# mini-projeto de analise de sentimentos
### dataset: https://www.kaggle.com/datasets/kavita5/twitter-dataset-avengersendgame

## Importando as ferramentas

In [86]:
import pandas as pd
import seaborn as sns
import re

In [ ]:
# se vc n tem a nltk instalada
!pip install nltk
import nltk
nltk.download()

## olhando os dados e fazendo ajustes iniciais

In [ ]:
df = pd.read_csv('./tweets.csv')
df.head()

In [ ]:
df.info()

In [ ]:
ds = df.loc[:, ['text', 'created', 'screenName', 'id']]
ds.loc['created'] = pd.to_datetime(df['created'])

In [ ]:
ds['text'] = ds['text'].astype('string')
ds['id'] = ds['id'].astype('int64')

In [83]:
ds.dropna(inplace=True)
print(f"número de linhas: {ds.shape[0]}")
ds.head()

número de linhas: 15000


,text,created,screenName,id
0,RT @mrvelstan: literally nobody: me: #Avenger...,2019-04-23 10:43:30,DavidAc96,1120639328034680064
1,"RT @agntecarter: i�m emotional, sorry!! 2014 ...",2019-04-23 10:43:30,NRmalaa,1120639325199200000
2,saving these bingo cards for tomorrow � #Ave...,2019-04-23 10:43:30,jijitsuu,1120639324683289984
3,RT @HelloBoon: Man these #AvengersEndgame ads ...,2019-04-23 10:43:29,SahapunB,1120639323328540032
4,"RT @Marvel: We salute you, @ChrisEvans! #Capta...",2019-04-23 10:43:29,stella22_97,1120639321571069952


## pegando uma amostra e iniciando a analise

In [105]:
work = ds['text'].sample(n=1000)
work = work.str.lower()

In [106]:
work.iloc[:5]

3911     rt @ashchanchlani: after party #avengersendgam...
11964    rt @marvel: 44,800 #lego bricks. 270 hours. on...
14081    rt @mcu_direct: the first non-spoiler #avenger...
4770     rt @cevansofficial: new: @chrisevans and scarl...
7670     rt @marvelstudios: the stars of #avengersendga...
Name: text, dtype: string

In [109]:
# usando regex e funções do python para retirar/substituir padrões indesejados

# substituindo cacacteres desconhecidos por nada(apagando)
work = work.apply(lambda s: re.sub(r'[^\x00-\x7f]','', s))
# substituindo a string 'rt @' por nada(apagando)
work = work.apply(lambda s: s.replace('rt @', ''))
#removendo todos os caracteres que não sejam letras do alfabeto e os que formam links (padrão http)
work = work.apply(lambda s: re.sub(r"\s|[^a-zA-Z]|http\S+",' ', s))
# removendo os espaços desnecessários
work = work.apply(lambda s: ' '.join(s.split()))

work.iloc[:5]

3911     ashchanchlani after party avengersendgame u u ...
11964    marvel lego bricks hours one heroic set from l...
14081    mcu direct the first non spoiler avengersendga...
4770     cevansofficial new chrisevans and scarlett joh...
7670     marvelstudios the stars of avengersendgame beg...
Name: text, dtype: object

In [117]:
# tirando termos redundantes (stopwords) + reduzindo os termos aos seus radicais (stemming)

stopwords = nltk.corpus.stopwords.words("english")
stemmer = nltk.stem.PorterStemmer()
tokenizer = nltk.tokenize.word_tokenize

def filtragem(string):
    tokens = tokenizer(string)
    newStr = [stemmer.stem(token) for token in tokens if (token not in stopwords)]
    return ' '.join(newStr)

work = work.apply(filtragem)

In [ ]:
# analisando quais os termos mais falados